# Make efficiency map plots for trained fitters

In [1]:
from ipywidgets import interact, fixed, interactive, widgets, interact_manual
from IPython.display import display, clear_output, HTML



import train as tn
reload(tn)

import plotting
reload(plotting)

import util as ut

import matplotlib.pyplot as plt
from matplotlib import ticker
plt.style.use('seaborn-poster')
%matplotlib inline

import numpy as np

from pprint import pprint

import itertools
import time

import os

from collections import OrderedDict

Welcome to ROOTaaS 6.06/08


In [2]:
def GetFitter ( dataDir, 
                inputName, 
                inputDir,
                load
              ) :
    
    #---------------------------------------------------------------------------- 
    if (load) :
        w_load_bar = widgets.IntProgress(
        min=0,
        max=3,
        description='Loading:',
        bar_style='success',
        orientation='horizontal' )   
        
        display(w_load_bar)
            
        ut.defaultParameters(dataDir=dataDir, inputName=inputName, inputDir=inputDir)
        w_load_bar.value += 1
        ut.setParams()
        w_load_bar.value += 1
        effFitter = ut.loadOrMake()
        w_load_bar.value += 1

        time.sleep(1)
        print('Done loading')
        w_load_bar.close()

        return effFitter
    #----------------------------------------------------------------------------

In [3]:
def GetDictionary (inputDir) :
    #----------------------------------------------------------------------------
    classifier_names = []
    try :
        for names in os.listdir(inputDir) :
            if names.endswith(".pkl.gz") :
                classifier_names.append(names[:-7])
        if (classifier_names != []) :
            classifier_dictionary = dict(zip(classifier_names,classifier_names))
            return classifier_dictionary, inputDir;
        else : 
            print('No classifiers in directory '+inputDir)
            return 'Failure';
    except OSError :
        print("No such directory")
        return 'Failure';
    #----------------------------------------------------------------------------  
    

In [4]:
input_Dir = widgets.Text(
    value='./classifiers',
    placeholder='directory to the classifiers',
    description='inputDir:',
    disabled=False,
)


data_Dir = widgets.Text(
    value='./data',
    placeholder='directory to data files',
    description='dataDir:',
    disabled=False
)

In [5]:
w = interactive(GetDictionary,inputDir=input_Dir, display=False)
display(w)
#clear_output()

({u'Train_Configurable_Refactor_higgs_model_dep_1702482': u'Train_Configurable_Refactor_higgs_model_dep_1702482',
  u'Train_Configurable_Refactor_higgs_model_dep_1735132': u'Train_Configurable_Refactor_higgs_model_dep_1735132',
  u'effGenVarClassBinnedRecoPtBinnedRecoNjets2p5_out': u'effGenVarClassBinnedRecoPtBinnedRecoNjets2p5_out',
  u'effGenVarClassBinnedRecoPt_out': u'effGenVarClassBinnedRecoPt_out',
  u'effGenVarClass_binnedPtNjets_out': u'effGenVarClass_binnedPtNjets_out',
  u'effGenVarClass_binnedRecoPt_out': u'effGenVarClass_binnedRecoPt_out',
  u'fullBDT': u'fullBDT'},
 u'./classifiers')

In [6]:
#extract the dictionary with the classifier names found in the
#given directory plus the directory name
class_dict = w.result[0]
inDir = w.result[1]

In [7]:
w_input_Name = widgets.Dropdown(
    description='Classifier:',
    options=class_dict,
    )
w_Load = widgets.Checkbox(
    value=False,
    description='Load classifier',
    disabled=False
)
HTML('<style> .widget-hbox .widget-label { max-width:350ex; text-align:left} </style>')

In [8]:
fitterObject = interactive(GetFitter,
                           inputName=w_input_Name,
                            dataDir =data_Dir,
                            inputDir=fixed(inDir),
                            load = w_Load
                          )

display(fitterObject)
clear_output()

entered config files named my_train_config
None
Load object with the name fullBDT and the following paramters 
./classifiers
./classifiers/fullBDT.pkl.gz
loading pickle ./classifiers/fullBDT.pkl.gz
loading data ./classifiers/fullBDT.root
Index([u'absweight', u'class', u'genJet2p5Pt0', u'genJet2p5Pt1',
       u'genJet2p5Pt2', u'genJet2p5Pt3', u'genJet2p5Rapidity0',
       u'genJet2p5Rapidity1', u'genJet2p5Rapidity2', u'genJet2p5Rapidity3',
       u'genNjets2p5', u'genPt', u'genRapidity', u'recoNjets2p5', u'recoPt',
       u'recoRapidity', u'weight', u'proc', u'absGenRapidity',
       u'absGenJet2p5Rapidity0', u'absGenJet2p5Rapidity1',
       u'absGenJet2p5Rapidity2', u'absGenJet2p5Rapidity3', u'recoPtBin',
       u'recoPtCat', u'recoNjets2p5Bin', u'recoNjets2p5Cat', u'class_prob_0',
       u'class_prob_1', u'class_prob_2', u'class_prob_3', u'recoPtCat_prob_0',
       u'recoPtCat_prob_1', u'recoPtCat_prob_2', u'recoPtCat_prob_3',
       u'recoPtCat_prob_4', u'recoPtCat_prob_5', u'recoPtC

## projection on the following 2 variables

In [9]:
proj_vars = ['absGenJet2p5Rapidity0','absGenJet2p5Rapidity1','absGenJet2p5Rapidity2',
    'absGenJet2p5Rapidity3','absGenRapidity','genJet2p5Pt0','genJet2p5Pt1','genJet2p5Pt2',
             'genJet2p5Pt3','genPt']
proj_keys = ['|y| leading jet','|y| subleading jet','|y| 3rd leading jet','|y| 4th leading jet','|y| di-photon',
            'pt leading jet','pt subleading jet','pt 3rd leading jet','pt 4th leading jet','pt di-photon']

proj_var_dict = OrderedDict(zip(proj_keys,proj_vars))

In [10]:
w_varName_x = widgets.Dropdown(
    options=proj_var_dict,
    description='x-axis:'
)

w_varName_y = widgets.Dropdown(
    options=proj_var_dict,
    description='y-axis:',
)

w_mres_cat = widgets.ToggleButtons(
    options={'bad':0, 'medium':1, 'good':2},
    description='Di-photon mass resolution:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
#     icon='check'
)

w_noJets = widgets.SelectionSlider(
    options=OrderedDict(zip(['0','1','2','3','>3'],[0,1,2,3,4])),
    description='Number of jets:',
)

w_detector_eff = widgets.Checkbox(
    value=False,
    description='detector efficiency',
    disabled=False
)

In [11]:
def CheckforPt (name) :
    if 'Pt' in name :
        return True
    return False

In [12]:
def NjetsEffPlots (effFitter, x_var, y_var,m_gamma_cat=0, Njets=0, effTag=False) :
    
    df = effFitter.df
    first_train_evt = int(round(df.index.size*(1.-effFitter.split_frac)))
    #take the test sample 
    df_test = df[:first_train_evt]
    
    #define bins
    print(CheckforPt(x_var))
    print(CheckforPt(y_var))
    defineBins = GetBins(x_pt=CheckforPt(x_var),y_pt=CheckforPt(y_var),x_name=x_var,y_name=y_var)
    ut.runDefineBins(effFitter,defineBins)
    
    print(defineBins)
    
    if effTag :
        NjetsCat = 0
    else :
        NjetsCat = 3*Njets +1 + m_gamma_cat
    
    i = NjetsCat
    print(i)
    
    column_proba_name = 'recoNjets2p5Cat_prob_'+str(i)
    
    gb_freq  = df.groupby([x_var+'Bin',y_var+'Bin']).apply(weight_freq,"recoNjets2p5Cat",(i-1),'absweight')
    gb_proba = df_test.groupby([x_var+'Bin',y_var+'Bin']).apply(weighted_average, column_proba_name,'absweight')
    
    display(gb_proba[0:5])
    
    title1 ='recoNjetsCat('+str(i-1)+') \n predicted by clf'
    title2 ='recoNjetsCat('+str(i-1)+') \n true from data'
   

      
    if (i==0) :
        print('perform efficiency (1- reco-proba) plot')
        plot_imshow([gb_proba,gb_freq],binBoundaries=defineBins,x_lab=x_var,y_lab=y_var,titles=['predicted (clf) reco eff',
                                                                    'true data reco eff'],effTag=True)
    else :
        plot_imshow([gb_proba,gb_freq],binBoundaries=defineBins, x_lab=x_var,y_lab=y_var,titles=[title1,title2])
    
    


In [13]:
def GetBins (x_pt, y_pt, x_name, y_name) :
    N_PtBins = 8
    N_RapidityBins = 8
    if (x_pt and y_pt) :
        return { x_name : dict(boundaries=np.linspace(0,300.,N_PtBins)), 
                       y_name : dict(boundaries=np.linspace(0,300.,N_PtBins))
            }   
    else :
        if x_pt :
            return { x_name : dict(boundaries=np.linspace(0,300.,N_PtBins)), 
                           y_name : dict(boundaries=np.linspace(0.,2.5,N_RapidityBins))
                          }
        if y_pt :
            return { x_name : dict(boundaries=np.linspace(0.,2.5,N_RapidityBins)), 
                           y_name : dict(boundaries=np.linspace(0,300.,N_PtBins))
                          }
    return { x_name : dict(boundaries=np.linspace(0.,2.5,N_RapidityBins)), 
             y_name : dict(boundaries=np.linspace(0.,2.5,N_RapidityBins))
                        }

### Extract data frame and start getting the probas in a binned way

In [14]:
def weighted_average(df_name, column_name, weight_name=None):
    """
    This function computes the weighted average of the quantity column_name
    stared in the pandas dataframe df_name. In case no weights are given
    or if they sum up to zero, the mean is returned instead.
    :params 
            df_name :
        column_name :
        weight_name :
    :retruns
                    :
    """
    d = df_name[column_name]
    w = df_name[weight_name]
    #print(len(w))
    if (weight_name == None) :
        return float(d.mean())
    else :
        try:
            return (d * w).sum() / float(w.sum())
        except ZeroDivisionError:
            return float(d.mean())

In [15]:
def weight_freq (df_name, column_name, equal_to, weight_name) :
    df = df_name#[df_name[column_name]==equal_to]
    
    w_all = df[weight_name].sum()
    w_PartPhaseSpace = df[df[column_name] == equal_to][weight_name].sum()
    #print(w_all)
    #print(w_PartPhaseSpace)
    return w_PartPhaseSpace / w_all

In [16]:
def plot_imshow(groupby_objects, binBoundaries, 
                          titles=[],
                        x_lab = None,
                        y_lab = None,
                          cmap=plt.cm.Blues,
                   effTag=False, effPlot=False):
   

    
    if effTag :
        plot_imshow(groupby_objects, binBoundaries=binBoundaries, titles=['predicted (clf) reco eff', 'true (data) reco eff'],
                        x_lab = x_lab, y_lab = y_lab, cmap=plt.cm.Reds, effTag=False, effPlot=True)
    else :
        
        cm_list = []

        for k, r in enumerate(groupby_objects) :
            x,y = r.index.levels
            print(len(x))
            print(len(y))
            print(np.shape(r.values))
            cm = r.values.reshape(len(x),len(y))
            if effPlot :
                cm = 1. - cm
            
            #check this
            cm_list.append(cm.T)    
                        
        
        fig, axarr = plt.subplots(1,2,figsize=(10,10))
        
        
        minimum = np.amin(cm_list)
        maximum = np.amax(cm_list)
        
        for l, cm in enumerate(cm_list) :
        
            plt.subplot('22'+str(l+1))
            
            
            plt.imshow(cm_list[l], interpolation='nearest', cmap=cmap,origin='lower',
                       vmin=minimum,vmax=maximum)
            
            plt.title(titles[l])

            xtick_marks = np.arange(len(x)+1)-0.5
            xtick_names = binBoundaries[x_lab]['boundaries']
            ytick_marks = np.arange(len(y))-0.5
            ytick_names = binBoundaries[y_lab]['boundaries']
            
            xtick_names = np.round(xtick_names,2)
            ytick_names = np.round(ytick_names,2)
            plt.xticks(xtick_marks,xtick_names,rotation='90')
            
            
            
            

            plt.xlabel(x_lab)
            if l == 0 :
                print('hi')
                plt.ylabel(y_lab)
                plt.yticks(ytick_marks, ytick_names)
            else :
                #bad trick
                plt.yticks(visible=False)
            
            #plt.tight_layout()

            # text
            thresh = (maximum + minimum) / 2.
            for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
                plt.text(j, i, '%.2f' % cm[i, j], horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        
        fig.subplots_adjust(top=0.85)   
        cbar_ax = fig.add_axes([0.1, 0.98, 0.8, 0.02])
        plt.colorbar(cax=cbar_ax,orientation='horizontal')#,norm=plt.colors.Normalize(vmin=min_col,vmax=max_col))
        
        plt.show()
        if l == 0 :
            plotRelDiff_imshow(groupby_objects=groupby_objects,binBoundaries=binBoundaries,x_lab=x_lab,y_lab=y_lab,cmap=plt.cm.Oranges,effPlot=True)
        else :
            plotRelDiff_imshow(groupby_objects=groupby_objects,binBoundaries=binBoundaries,x_lab=x_lab,y_lab=y_lab,cmap=plt.cm.Oranges)

                 
    

In [17]:
def plotRelDiff_imshow(groupby_objects,binBoundaries, 
                        x_lab = None,
                        y_lab = None,
                          cmap=plt.cm.Blues,
                      effPlot=False) :
   
        
    fig = plt.figure(figsize=(7,7))

    r_pred, r_true = groupby_objects

    x_pred,y_pred = r_pred.index.levels
    x_true,y_true = r_true.index.levels

    cm_pred = r_pred.values.reshape(len(x_pred),len(y_pred))
    cm_true = r_true.values.reshape(len(x_true),len(y_true))

    if effPlot :
        cm_true = 1. - cm_true
        cm_pred = 1. - cm_pred
    
    #plt.subplot(111)
    total_sum_true_permille = np.sum(np.sum(cm_true))/100.
    print(total_sum_true_permille)
    
    cm = np.divide((cm_true-cm_pred),cm_true, out=np.zeros_like(cm_true-cm_pred), 
                   where=cm_true>total_sum_true_permille ) 
    
    cm = cm.T
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap,origin='lower')
    plt.title(r'$\frac{\mathrm{true} - \mathrm{pred}}{\mathrm{true}}$'+ '\n')
    
    
    
    xtick_marks = np.arange(len(x_pred)+1)-0.5
    xtick_names = binBoundaries[x_lab]['boundaries']
    ytick_marks = np.arange(len(y_pred))-0.5
    ytick_names = binBoundaries[y_lab]['boundaries']

    xtick_names = np.round(xtick_names,2)
    ytick_names = np.round(ytick_names,2)
    plt.xticks(xtick_marks,xtick_names,rotation='90')

    plt.xticks(xtick_marks, xtick_names)
    plt.yticks(ytick_marks, ytick_names)


    thresh = (cm.max()+cm.min()) / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, np.round(cm[i, j],2), horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")


    plt.xlabel(x_lab)
    plt.ylabel(y_lab)
    plt.tight_layout()

    fig.subplots_adjust(top=0.78)   
    cbar_ax = fig.add_axes([0.15, 0.97, 0.7, 0.02])
    cb = plt.colorbar(cax=cbar_ax,orientation='horizontal')#,norm=plt.colors.Normalize(vmin=min_col,vmax=max_col))

    #tick_locator = ticker.MaxNLocator(nbins=6)
    #cb.locator = tick_locator
    #cb.update_ticks()
       
    

#### Perform plots with realtive ratio of true and predicted probas:  $ \frac{\text{true} - \text{pred}}{\text{true}} $

In [19]:
fitterObject.result

In [20]:

#interact(disable(w_detector_eff,[w_noJets,w_mres_cat]))
#display(w_detector_eff)
#display(w_noJets)
#display(w_mres_cat)
interact_manual(NjetsEffPlots,effFitter=fixed(fitterObject.result),x_var=w_varName_x,y_var=w_varName_y,
              m_gamma_cat=w_mres_cat, Njets=w_noJets, effTag=w_detector_eff)


False
False
[ 0.          0.35714286  0.71428571  1.07142857  1.42857143  1.78571429
  2.14285714  2.5       ]
[ 0.          0.35714286  0.71428571  1.07142857  1.42857143  1.78571429
  2.14285714  2.5       ]
{'absGenJet2p5Rapidity0': {'boundaries': array([ 0.        ,  0.35714286,  0.71428571,  1.07142857,  1.42857143,
        1.78571429,  2.14285714,  2.5       ])}, 'absGenJet2p5Rapidity2': {'boundaries': array([ 0.        ,  0.35714286,  0.71428571,  1.07142857,  1.42857143,
        1.78571429,  2.14285714,  2.5       ])}}
2


KeyError: 'absGenJet2p5Rapidity2Bin'